In [7]:
!pip install sentence-transformers scikit-learn pandas


In [9]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.3 MB/s eta 0:00:00


In [11]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
from google.colab import files
uploaded = files.upload()

Saving resume1.pdf to resume1.pdf
Saving resume2.pdf to resume2.pdf
Saving resume3.pdf to resume3.pdf
Saving resume4.pdf to resume4.pdf
Saving resume5.pdf to resume5.pdf
Saving resume6.pdf to resume6.pdf
Saving resume7.pdf to resume7.pdf
Saving resume8.pdf to resume8.pdf
Saving resume9.pdf to resume9.pdf
Saving resume10.pdf to resume10.pdf
Saving resume11.pdf to resume11.pdf
Saving resume12.pdf to resume12.pdf
Saving resume13.pdf to resume13.pdf
Saving resume14.pdf to resume14.pdf
Saving resume15.pdf to resume15.pdf


In [14]:
import pdfplumber

resume_texts = []
resume_names = []

for file in uploaded.keys():
    text = ""
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + " "

    resume_texts.append(text)
    resume_names.append(file)

print("Total resumes loaded:", len(resume_texts))

Total resumes loaded: 15


In [15]:
job_description = """
We are hiring a Backend Python Developer with experience in:
- Python
- Django
- REST APIs
- PostgreSQL
- Docker
- AWS
- Git
"""

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
resume_embeddings = model.encode(resume_texts)
jd_embedding = model.encode([job_description])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

similarities = cosine_similarity(resume_embeddings, jd_embedding).flatten()

results = pd.DataFrame({
    "Resume": resume_names,
    "Similarity Score": similarities
}).sort_values(by="Similarity Score", ascending=False)

results

,Resume,Similarity Score
0,resume1.pdf,0.669353
3,resume4.pdf,0.507585
13,resume14.pdf,0.445949
9,resume10.pdf,0.429299
1,resume2.pdf,0.414659
10,resume11.pdf,0.404885
6,resume7.pdf,0.400952
5,resume6.pdf,0.391969
14,resume15.pdf,0.382421
11,resume12.pdf,0.368607


In [20]:
jd_skills = [
    "python",
    "django",
    "rest",
    "postgresql",
    "docker",
    "aws",
    "git"
]

In [21]:
skill_results = []

for text in resume_texts:
    text_lower = text.lower()

    matched = []
    missing = []

    for skill in jd_skills:
        if skill in text_lower:
            matched.append(skill)
        else:
            missing.append(skill)

    overlap_percentage = (len(matched) / len(jd_skills)) * 100

    skill_results.append({
        "Matched Skills": ", ".join(matched),
        "Missing Skills": ", ".join(missing),
        "Skill Overlap %": overlap_percentage
    })

In [22]:
explain_df = pd.DataFrame(skill_results)

final_results = pd.concat([results.reset_index(drop=True), explain_df], axis=1)

final_results

,Resume,Similarity Score,Matched Skills,Missing Skills,Skill Overlap %
0,resume1.pdf,0.669353,"python, django, rest, postgresql, docker, aws,...",,100.000000
1,resume4.pdf,0.507585,git,"python, django, rest, postgresql, docker, aws",14.285714
2,resume14.pdf,0.445949,python,"django, rest, postgresql, docker, aws, git",14.285714
3,resume10.pdf,0.429299,"docker, aws","python, django, rest, postgresql, git",28.571429
4,resume2.pdf,0.414659,"rest, git","python, django, postgresql, docker, aws",28.571429
5,resume11.pdf,0.404885,"python, docker, aws","django, rest, postgresql, git",42.857143
6,resume7.pdf,0.400952,"docker, aws, git","python, django, rest, postgresql",42.857143
7,resume6.pdf,0.391969,"rest, git","python, django, postgresql, docker, aws",28.571429
8,resume15.pdf,0.382421,"python, aws","django, rest, postgresql, docker, git",28.571429
9,resume12.pdf,0.368607,"rest, git","python, django, postgresql, docker, aws",28.571429


In [23]:

final_results["Normalized Similarity"] = (
    final_results["Similarity Score"] - final_results["Similarity Score"].min()
) / (
    final_results["Similarity Score"].max() - final_results["Similarity Score"].min()
)

final_results["Normalized Overlap"] = final_results["Skill Overlap %"] / 100


final_results["Final Score"] = (
    0.7 * final_results["Normalized Similarity"] +
    0.3 * final_results["Normalized Overlap"]
)


final_results = final_results.sort_values(by="Final Score", ascending=False)

final_results[["Resume", "Similarity Score", "Skill Overlap %", "Final Score"]]

,Resume,Similarity Score,Skill Overlap %,Final Score
0,resume1.pdf,0.669353,100.000000,1.000000
1,resume4.pdf,0.507585,14.285714,0.483762
5,resume11.pdf,0.404885,42.857143,0.404986
3,resume10.pdf,0.429299,28.571429,0.401232
6,resume7.pdf,0.400952,42.857143,0.398687
2,resume14.pdf,0.445949,14.285714,0.385042
4,resume2.pdf,0.414659,28.571429,0.377784
7,resume6.pdf,0.391969,28.571429,0.341442
8,resume15.pdf,0.382421,28.571429,0.326150
9,resume12.pdf,0.368607,28.571429,0.304024


In [27]:
!pip install gradio --quiet

In [28]:
import gradio as gr
import pdfplumber
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def screen_resumes(files, job_description):

    model = SentenceTransformer('all-MiniLM-L6-v2')

    resume_texts = []
    resume_names = []

    for file in files:
        text = ""
        with pdfplumber.open(file.name) as pdf:
            for page in pdf.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + " "

        resume_texts.append(text)
        resume_names.append(file.name)

    resume_embeddings = model.encode(resume_texts)
    jd_embedding = model.encode([job_description])

    similarities = cosine_similarity(resume_embeddings, jd_embedding).flatten()

    jd_skills = ["python", "django", "rest", "postgresql", "docker", "aws", "git"]

    skill_overlap = []
    for text in resume_texts:
        text_lower = text.lower()
        matched = [skill for skill in jd_skills if skill in text_lower]
        overlap = len(matched) / len(jd_skills)
        skill_overlap.append(overlap)

    sim_norm = (similarities - similarities.min()) / (similarities.max() - similarities.min())

    final_scores = 0.7 * sim_norm + 0.3 * np.array(skill_overlap)

    results = pd.DataFrame({
        "Resume": resume_names,
        "Similarity Score": similarities,
        "Skill Overlap %": np.array(skill_overlap) * 100,
        "Final Score": final_scores
    }).sort_values(by="Final Score", ascending=False)

    return results


interface = gr.Interface(
    fn=screen_resumes,
    inputs=[
        gr.File(file_count="multiple", file_types=[".pdf"]),
        gr.Textbox(label="Paste Job Description Here", lines=6)
    ],
    outputs=gr.Dataframe(),
    title="AI Resume Screening System",
    description="Upload multiple resumes and paste job description to rank candidates."
)

interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f99c018ed2284e414.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
